## 웹스크래핑을 이용한 데이터 수집

* open API를 이용한 데이터 수집이 불가한 경우 해당
* 웹을 요청하여 html을 모두 받아서 DOM으로 올려서 데이터를 추출방식
* DOM을 띠워서 데이터를 추출할때 사용하는 라이브러리
 > beautifulsoup (bs4)를 활용  
 > conda install beautifulsoup4

In [23]:
from bs4 import BeautifulSoup

In [24]:
from urllib.request import urlopen

- 콘텐츠가 존재하는 해당 페이지까지 진입
- 진입간에 로그인, ajax등등 사람의 손을 타지 않는지 체크
- 그냥 url만 넣으면 화면이 구성된다 => OK
- html 자체에 프레임이 적용된경우 실제 주소까지 찾아서 이동
- 통신시 get, post등 데이터를 전달해서 획득하는 것도 OK

### 네이버 금융 > 고시환율정보 수집

- 하루에 환율이 수시로 변경된다 
- 그 주기를 관찰하여 수집 주기를 결정해야 한다

In [25]:
target_site = 'https://finance.naver.com/marketindex/exchangeList.nhn'
target_site

'https://finance.naver.com/marketindex/exchangeList.nhn'

In [26]:
#요청 응답 확득하기 
page = urlopen(target_site)
page

In [27]:
# DOM  구성
# 'html5lib' 파서는 html  양이 크거나, 정교한 파싱을 할때 즉, 아래 파서가
# 정상적으로 결과를 내지 못하면 이 파서로 교체
soup = BeautifulSoup(page, 'html.parser' )

In [28]:
#리스트를 내포를 통해서 간단하게 가공해 보았다.
tmp = [td.a.string.strip() for td in soup.find_all('td','tit')]
tmp[:5]

['미국 USD', '유럽연합 EUR', '일본 JPY (100엔)', '중국 CNY', '홍콩 HKD']

In [38]:
for td in soup.find_all('td','tit'):
    print(td.string.strip())

미국 USD
유럽연합 EUR
일본 JPY (100엔)
중국 CNY
홍콩 HKD
대만 TWD
영국 GBP
오만 OMR
캐나다 CAD
스위스 CHF
스웨덴 SEK
호주 AUD
뉴질랜드 NZD
체코 CZK
칠레 CLP
터키 TRY
몽골 MNT
이스라엘 ILS
덴마크 DKK
노르웨이 NOK
사우디아라비아 SAR
쿠웨이트 KWD
바레인 BHD
아랍에미리트 AED
요르단 JOD
이집트 EGP
태국 THB
싱가포르 SGD
말레이시아 MYR
인도네시아 IDR 100
카타르 QAR
카자흐스탄 KZT
브루나이 BND
인도 INR
파키스탄 PKR
방글라데시 BDT
필리핀 PHP
멕시코 MXN
브라질 BRL
베트남 VND 100
남아프리카 공화국 ZAR
러시아 RUB
헝가리 HUF
폴란드 PLN


In [29]:
for td in soup.find_all('td','tit'):
    print(td.a.string.strip())

미국 USD
유럽연합 EUR
일본 JPY (100엔)
중국 CNY
홍콩 HKD
대만 TWD
영국 GBP
오만 OMR
캐나다 CAD
스위스 CHF
스웨덴 SEK
호주 AUD
뉴질랜드 NZD
체코 CZK
칠레 CLP
터키 TRY
몽골 MNT
이스라엘 ILS
덴마크 DKK
노르웨이 NOK
사우디아라비아 SAR
쿠웨이트 KWD
바레인 BHD
아랍에미리트 AED
요르단 JOD
이집트 EGP
태국 THB
싱가포르 SGD
말레이시아 MYR
인도네시아 IDR 100
카타르 QAR
카자흐스탄 KZT
브루나이 BND
인도 INR
파키스탄 PKR
방글라데시 BDT
필리핀 PHP
멕시코 MXN
브라질 BRL
베트남 VND 100
남아프리카 공화국 ZAR
러시아 RUB
헝가리 HUF
폴란드 PLN


- 데이터를 추출하여 디비에 적제하기 위해 최종 형태는 다음과 같다
- [
    {
      'name':'미국 USD',
      'code';'USD',
      'buy_std_rate':1209.90,
      'cash_sell':1188.73
    },{}
 ]

In [30]:
# replace를 이용해 콤마를 삭제 ('기존','변경')
'1,234'.replace(',','')

'1234'

In [31]:
# 통화 
tmp = [td.a.string.strip() for td in soup.find_all('td','tit')]
tmp[:3]

['미국 USD', '유럽연합 EUR', '일본 JPY (100엔)']

In [32]:
# 매매기준율
tmp = [td.string.strip() for td in soup.find_all('td','sale')]
tmp[:3]

['1,194.50', '1,314.79', '1,109.31']

```
body > div > table > tbody > tr:nth-child(1) > td:nth-child(4)
```
- 만약 대상 페이지에 table이 n개 존재하면 특정해서 찾아야한다
- 대상 데이터가 있는 tr을 모두 찾았다
- table.tbl_exchange > tbody > tr
- 위의 표현을 for문으로 구동하고 하나하나의 tr에서 td:nth-child(4) 를 선택


In [33]:
body > div > table > tbody > tr:nth-child(1) > td:nth-child(4)

SyntaxError: invalid syntax (<ipython-input-33-0a853f966b29>, line 1)

In [81]:
# 원하는 데이터를 감사는 tr를 찾아서 그밑에서 자식들을 탐색후 거기서 데이터를 추출!!
for tr in soup.select('table.tbl_exchange > tbody > tr'):
    # 데이터 한줄 한줄 뽑아서 => tr에서 탐색하여 세부 데이터 추출
    print( tr.select_one('td.tit').string.strip() )
    print( tr.select_one('td.sale').string.strip() )
    print( tr.select('td')[3].string.strip())

미국 USD
1,194.50
1,173.60
유럽연합 EUR
1,314.79
1,288.63
일본 JPY (100엔)
1,109.31
1,089.90
중국 CNY
167.98
159.59
홍콩 HKD
152.37
149.37
대만 TWD
38.36
35.68
영국 GBP
1,475.75
1,446.68
오만 OMR
3,102.44
2,916.30
캐나다 CAD
909.06
891.16
스위스 CHF
1,202.44
1,178.76
스웨덴 SEK
123.35
120.33
호주 AUD
820.32
804.16
뉴질랜드 NZD
766.63
751.53
체코 CZK
50.83
46.26
칠레 CLP
1.68
1.55
터키 TRY
207.07
0.00
몽골 MNT
0.45
0.00
이스라엘 ILS
336.87
309.93
덴마크 DKK
176.23
171.92
노르웨이 NOK
133.06
129.81
사우디아라비아 SAR
318.47
296.50
쿠웨이트 KWD
3,928.50
3,614.22
바레인 BHD
3,167.93
2,914.50
아랍에미리트 AED
325.20
302.77
요르단 JOD
1,684.77
1,549.99
이집트 EGP
72.81
0.00
태국 THB
39.07
36.73
싱가포르 SGD
865.86
848.63
말레이시아 MYR
285.80
264.66
인도네시아 IDR 100
8.52
7.67
카타르 QAR
328.07
0.00
카자흐스탄 KZT
3.09
0.00
브루나이 BND
865.83
813.89
인도 INR
16.66
0.00
파키스탄 PKR
7.62
0.00
방글라데시 BDT
14.14
0.00
필리핀 PHP
22.92
21.05
멕시코 MXN
61.21
56.07
브라질 BRL
292.69
269.28
베트남 VND 100
5.15
4.55
남아프리카 공화국 ZAR
81.25
74.75
러시아 RUB
18.26
16.26
헝가리 HUF
3.96
3.65
폴란드 PLN
303.41
279.14


In [83]:
results = []
for tr in soup.select('table.tbl_exchange > tbody > tr'):
    if tr.select_one('td.tit').a.string.strip().count('JPY') == 0:
        tmp   = dict()
        tmp['Name']         = tr.select_one('td.tit').a.string.strip()
        tmp['Code']         = tr.select_one('td.tit').a['href'][-6:-3]
        tmp['Buy_std_rate'] = tr.select_one('td.sale').string.strip() 
        tmp['Cash_sell']    = tr.select('td')[3].string.strip()
        # :nth-child(4)가 않되는 분은 아래로 처리변경
        #tmp['cash_sell']    = tr.select('td')[3].string.strip()
        results.append( tmp )
results

[{'Name': '미국 USD',
  'Code': 'USD',
  'Buy_std_rate': '1,194.50',
  'Cash_sell': '1,173.60'},
 {'Name': '유럽연합 EUR',
  'Code': 'EUR',
  'Buy_std_rate': '1,314.79',
  'Cash_sell': '1,288.63'},
 {'Name': '중국 CNY',
  'Code': 'CNY',
  'Buy_std_rate': '167.98',
  'Cash_sell': '159.59'},
 {'Name': '홍콩 HKD',
  'Code': 'HKD',
  'Buy_std_rate': '152.37',
  'Cash_sell': '149.37'},
 {'Name': '대만 TWD',
  'Code': 'TWD',
  'Buy_std_rate': '38.36',
  'Cash_sell': '35.68'},
 {'Name': '영국 GBP',
  'Code': 'GBP',
  'Buy_std_rate': '1,475.75',
  'Cash_sell': '1,446.68'},
 {'Name': '오만 OMR',
  'Code': 'OMR',
  'Buy_std_rate': '3,102.44',
  'Cash_sell': '2,916.30'},
 {'Name': '캐나다 CAD',
  'Code': 'CAD',
  'Buy_std_rate': '909.06',
  'Cash_sell': '891.16'},
 {'Name': '스위스 CHF',
  'Code': 'CHF',
  'Buy_std_rate': '1,202.44',
  'Cash_sell': '1,178.76'},
 {'Name': '스웨덴 SEK',
  'Code': 'SEK',
  'Buy_std_rate': '123.35',
  'Cash_sell': '120.33'},
 {'Name': '호주 AUD',
  'Code': 'AUD',
  'Buy_std_rate': '820.32',
  

In [84]:
results = [ {
    'name':tr.select_one('td.tit').a.string.strip(),
    'code':tr.select_one('td.tit').a['href'][-6:-3],
    'buy_std_rate':tr.select_one('td.sale').string.strip(),
    'cash_sell':tr.select('td')[3].string.strip()
} for tr in soup.select('table.tbl_exchange > tbody > tr') 
  if tr.select_one('td.tit').a.string.strip().count('JPY') == 0
]

In [85]:
results[:2]

[{'name': '미국 USD',
  'code': 'USD',
  'buy_std_rate': '1,194.50',
  'cash_sell': '1,173.60'},
 {'name': '유럽연합 EUR',
  'code': 'EUR',
  'buy_std_rate': '1,314.79',
  'cash_sell': '1,288.63'}]